In [1]:
!pip install -r requirements.txt


  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached httpcore-1.0.9-py3-none-any.whl.metadata (21 kB)
Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
Using cached httpcore-1.0.9-py3-none-any.whl (78 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 186.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 42.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 197.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 201.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 38.3 MB/s  0:00:08m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 72.1 MB/s  0:00:04m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 267.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 145.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 91.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/

In [2]:
from langchain_aws import ChatBedrockConverse
from langchain_core.documents import Document
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain_core.prompts import BasePromptTemplate
from langchain_core.output_parsers import BaseOutputParser
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from opensearchpy import OpenSearch, RequestsHttpConnection
import os
import boto3
import json
import sys
from typing import List, Optional, Dict, Tuple
from langchain_core.documents import Document
from pypdf import PdfReader
import concurrent.futures
from langchain_aws import BedrockEmbeddings
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import PromptTemplate
from operator import itemgetter
from langchain_core.runnables import RunnableLambda
from pydantic import BaseModel, Field
from sentence_transformers import SentenceTransformer
import torch


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/torch/cuda/__init__.py:61: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


In [3]:
device = 'cpu'
embedding_model = SentenceTransformer('all-MiniLM-L6-v2', device=device)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:

#text를 vector로 변환
def get_embedding(text):
    embeddings = embedding_model.encode(text)
    return embeddings


In [5]:
len(get_embedding("안녕하세요?"))

384

In [6]:
#opensearch에 index(sql table과 유사) 생성
def define_index(opensearch_client, index_name):
    index_settings = {
        "settings": {
            "index": {  
                "knn": True,                          
                "knn.algo_param.ef_search": 100,      
                "number_of_shards": 3,                
                "number_of_replicas": 2, 
                "analysis": {
                    "analyzer": {
                        "nori_analyzer": {
                            "tokenizer": "nori_tokenizer",
                            "filter": ["nori_stop", "lowercase"]
                        }
                    },
                    "filter": {
                        "nori_stop": {
                            "type": "nori_part_of_speech",
                            "stoptags": ["J", "JKS", "JKB", "JKO", "JKG", "JKC", "JKV", "JKQ", "JX", "JC"]
                        }
                    }
                }
            }
        },   
        "mappings": {
            "properties": {
                "document": {
                    "type": "text",
                    "analyzer": "nori_analyzer"
                },
                "doc_vector": {
                    "type": "knn_vector",
                    "dimension": 384,
                    "method": {
                        "name": "hnsw",   
                        "space_type": "l2",
                        "engine": "faiss",
                        "parameters": {
                            "ef_construction": 128,
                            "m": 16
                        }
                    }
                },
                "source": {
                    "type": "keyword"  # Use keyword for exact matching
                },
                "page": {
                    "type": "integer"
                },
                "total_pages": {
                    "type": "integer"
                }
            }
        }
    }
    try:
        response = opensearch_client.indices.create(
            index=index_name,
            body=index_settings
        )
        print("Index created successfully:", response)
        
    except Exception as e:
        print("Error creating index:", e)

In [7]:
#pdf파일 opensearch에 적재
def save_chunk(opensearh_client,  index_name,chunks):
    for d in enumerate(chunks):
        document = d[1].page_content
        raw_embedding = get_embedding(d[1].page_content)
        doc_vector = [float(val) for val in raw_embedding]
        source = d[1].metadata["source"]
        page_info= d[1].metadata["page"]
        total_page=d[1].metadata["total_pages"]
    
        # Prepare document for indexing
        document = {
            "document": document,
            "doc_vector":doc_vector,
            "source" : source, 
            "page" : page_info , 
            "total_pages" : total_page 
        }
        try:
            response = opensearh_client.index(
                index=index_name,
                body=document,
                id=f"sagemaker_doc_{d[0]}"
            )
            if d[0] % 500 ==0 :
                print(f"Document {d[0]} indexed successfully")
        except Exception as e:
            print(f"Error indexing document {d[0]}: {e}")


In [43]:
#속도를 위해 parallel하게 진행. 페이지를 부분적으로 나누어서 text파일 읽기
def process_page_range(pdf_path, start_page, end_page):
    """Process a range of pages from a PDF."""
    # Open the PDF
    doc = PdfReader(pdf_path)
    total_pages = len(doc.pages)  # Fix: use len(doc.pages)
    
    batch_docs = []
    for i in range(start_page, end_page):
        if i >= total_pages:
            break
            
        # Get page and extract text
        page = doc.pages[i]  # Fix: use doc.pages[i]
        text = page.extract_text()  # Fix: use extract_text()
        
        # Create Document object
        doc_obj = Document(
            page_content=text,
            metadata={
                "source": pdf_path,
                "page": i + 1,
                "total_pages": total_pages
            }
        )
        batch_docs.append(doc_obj)
    
    return batch_docs
    
#pdf파일 읽기 
def load_pdf_parallel(pdf_path, batch_size=100, max_workers=4):
    """Load a PDF using parallel processing."""
    # Get total pages
    doc = PdfReader(pdf_path)
    total_pages = len(doc.pages)  # Fix: use len(doc.pages)
    # Remove doc.close() - not needed for pypdf
    print(f"PDF has {total_pages} pages in total")
    
    # Create batches
    batches = []
    for start_page in range(0, total_pages, batch_size):
        end_page = min(start_page + batch_size, total_pages)
        batches.append((start_page, end_page))
    
    all_docs = []
    
    with concurrent.futures.ProcessPoolExecutor(max_workers=max_workers) as executor:
        # Submit tasks
        future_to_batch = {
            executor.submit(process_page_range, pdf_path, start, end): (start, end)
            for start, end in batches
        }
        
        # Process results as they complete
        for future in concurrent.futures.as_completed(future_to_batch):
            start, end = future_to_batch[future]
            try:
                batch_docs = future.result()
                all_docs.extend(batch_docs)
                print(f"Completed batch: pages {start+1} to {end}")
            except Exception as e:
                print(f"Error processing pages {start+1} to {end}: {e}")
    
    # Sort by page number
    all_docs.sort(key=lambda x: x.metadata["page"])
    
    return all_docs

In [44]:
#opensearh client 연결 생성 및 확인
opensearch_client = OpenSearch(
    hosts = [{'host': 'search-bnk-opensearch-domain-lmbhroa2z3ptbpsai3mf45jha4.ap-northeast-2.es.amazonaws.com', 'port': 443}],
    http_auth = ('Bnkadmin', 'Bnkadmin123!'),  # Replace with your master credentials
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection
)
try:
    response = opensearch_client.info()
    print("Successfully connected to OpenSearch")
    print(response)
except Exception as e:
    print(f"Connection failed: {e}")

Successfully connected to OpenSearch
{'name': '7dd5ae15fb2e9bf062b1cf97e77abe59', 'cluster_name': '010928186392:bnk-opensearch-domain', 'cluster_uuid': '55wiqk1PT-yFRCGPF6bwcw', 'version': {'number': '7.10.2', 'build_type': 'tar', 'build_hash': 'unknown', 'build_date': '2025-08-01T07:59:18.205170174Z', 'build_snapshot': False, 'lucene_version': '9.11.1', 'minimum_wire_compatibility_version': '7.10.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'The OpenSearch Project: https://opensearch.org/'}


In [45]:
index_name = "nori_test_index"

index_body = {
    "settings": {
        "analysis": {
            "analyzer": {
                "nori_analyzer": {
                    "type": "nori",
                    "decompound_mode": "mixed"
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "title": {
                "type": "text",
                "analyzer": "nori_analyzer"
            },
            "content": {
                "type": "text",
                "analyzer": "nori_analyzer"
            }
        }
    }
}

In [46]:
try:
    delete_response = opensearch_client.indices.delete(index=index_name)
    print(f"Index 's3_explain' deleted successfully: {delete_response}")
except Exception as delete_error:
    print(f"Failed to delete index 's3_explain': {delete_error}")
    
except Exception as e:
    print(f"Connection failed: {e}")

Index 's3_explain' deleted successfully: {'acknowledged': True}


In [47]:
# 인덱스 생성

opensearch_client.indices.create(index=index_name, body=index_body)


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'nori_test_index'}

In [48]:
# 2. 테스트 문서 인덱싱
test_docs = [
    {"title": "한국어 형태소 분석", "content": "노리 분석기는 한국어를 잘 처리합니다"},
    {"title": "은행 업무 시스템", "content": "금융권에서 사용하는 검색 시스템입니다"},
    {"title": "OpenSearch 활용", "content": "엘라스틱서치 호환 검색엔진입니다"}
]


In [49]:
for i, doc in enumerate(test_docs):
    opensearch_client.index(index=index_name, id=i+1, body=doc)



In [50]:
for i, doc in enumerate(test_docs):
    print(f"\n문서 {i+1}: {doc['title']}")
    print(f"원문: {doc['content']}")
    
    # 노리 analyzer로 분석
    analyze_result = opensearch_client.indices.analyze(
        index=index_name,
        body={
            "analyzer": "nori_analyzer",
            "text": doc['content']
        }
    )
    
    tokens = [token['token'] for token in analyze_result['tokens']]
    print(f"분석된 토큰: {' | '.join(tokens)}")







문서 1: 한국어 형태소 분석
원문: 노리 분석기는 한국어를 잘 처리합니다
분석된 토큰: 노리 | 분석기 | 분석 | 기 | 한국어 | 한국 | 어 | 처리

문서 2: 은행 업무 시스템
원문: 금융권에서 사용하는 검색 시스템입니다
분석된 토큰: 금융 | 사용 | 검색 | 시스템 | 입니다 | 이

문서 3: OpenSearch 활용
원문: 엘라스틱서치 호환 검색엔진입니다
분석된 토큰: 엘라스틱 | 서치 | 호환 | 검색 | 엔진 | 입니다 | 이


In [51]:


# 4. 검색 테스트
search_query = {
    "query": {
        "match": {
            "content": "한국어"
        }
    }
}

search_result = opensearch_client.search(index=index_name, body=search_query)
print(f"\n검색 결과: {search_result['hits']['total']['value']}건")




검색 결과: 0건


In [52]:
#pdf파일을 chunk로 나눈다
path="./s3-userguide.pdf"
page=load_pdf_parallel(path,max_workers=os.cpu_count())

PDF has 3031 pages in total


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Completed batch: pages 401 to 500
Completed batch: pages 501 to 600
Completed batch: pages 601 to 700
Completed batch: pages 301 to 400
Completed batch: pages 701 to 800
Completed batch: pages 201 to 300
Completed batch: pages 101 to 200
Completed batch: pages 801 to 900
Completed batch: pages 1001 to 1100
Completed batch: pages 901 to 1000
Completed batch: pages 1101 to 1200
Completed batch: pages 1201 to 1300
Completed batch: pages 1401 to 1500
Completed batch: pages 1301 to 1400
Completed batch: pages 1 to 100
Completed batch: pages 1501 to 1600
Completed batch: pages 1601 to 1700
Completed batch: pages 1901 to 2000
Completed batch: pages 1801 to 1900
Completed batch: pages 2001 to 2100
Completed batch: pages 2101 to 2200
Completed batch: pages 1701 to 1800
Completed batch: pages 2201 to 2300
Completed batch: pages 2401 to 2500
Completed batch: pages 2301 to 2400
Completed batch: pages 2501 to 2600
Completed batch: pages 2701 to 2800
Completed batch: pages 2801 to 2900
Completed bat

TOKENIZERS_PARALLELISM=(true | false)


Completed batch: pages 3001 to 3031


In [56]:
#반환된 결과는 List[Document] 형태입니다.
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=100)
split_docs = text_splitter.split_documents(page)


In [57]:
print("총 갯수 -> ",len(split_docs))

총 갯수 ->  8170


In [58]:
try:
    delete_response = opensearch_client.indices.delete(index='s3_explain')
    print(f"Index 's3_explain' deleted successfully: {delete_response}")
except Exception as delete_error:
    print(f"Failed to delete index 's3_explain': {delete_error}")
    
except Exception as e:
    print(f"Connection failed: {e}")

Failed to delete index 's3_explain': NotFoundError(404, 'index_not_found_exception', 'no such index [s3_explain]', s3_explain, index_or_alias)


In [59]:
#index 생성
define_index(opensearch_client, "s3_explain")

Index created successfully: {'acknowledged': True, 'shards_acknowledged': True, 'index': 's3_explain'}


In [63]:
split_docs[1000:1050]

[Document(metadata={'source': './s3-userguide.pdf', 'page': 340, 'total_pages': 3031}, page_content='때까지 업로드가 진행 중인 것으로 간주됩니다.\n멀티파트 업로드를 중지하려면 다음을 수행할 수 있습니다.\n1 S3Client 인스턴스를 만듭니다.\n2 버킷 이름과 기타 필수 파라미터를 전달하여 클라이언트의 중단 메서드를 사용합니\n다.\nNote\n특정 멀티파트 업로드를 중지할 수도 있습니다. 자세한 내용은 AWS SDK 사용(하위 수준 \nAPI) 섹션을 참조하세요.\nAWS SDK for Java를 사용하여 멀티파트 업로드를 중단하는 방법의 예는 Amazon S3 API 참조의\nCancel a multipart upload를 참조하세요.\n멀티파트 업로드 사용 API 버전 2006-03-01 319'),
 Document(metadata={'source': './s3-userguide.pdf', 'page': 341, 'total_pages': 3031}, page_content='Amazon Simple Storage Service 사용 설명서\n.NET\n다음 C# 예제는 1주일 이상 전에 특정 버킷에서 시작되어 진행 중인 멀티파트 업로드를 모두 중지\n합니다. 코드 예제 설정 및 실행에 대한 자세한 내용은 AWS SDK for .NET 개발자 안내서의 AWS \nSDK for .NET 시작하기를 참조하세요.\nusing Amazon;\nusing Amazon.S3;\nusing Amazon.S3.Transfer;\nusing System;\nusing System.Threading.Tasks;\nnamespace Amazon.DocSamples.S3\n{ \n    class AbortMPUUsingHighLevelAPITest \n    { \n        private const string bucketName = "*** provide bucket name **

In [60]:
save_chunk(opensearch_client, "s3_explain" ,  split_docs[1000:2500])

Document 0 indexed successfully
Document 500 indexed successfully
Document 1000 indexed successfully


In [64]:
#BM25 search ( nori_analyzer를 이용하여 형태소 분석을 진행한후 텍스트 유사도 측정 )
def simple_text_search(client, search_text, index_name="s3_explain", k=10):
    try:
        # Simple text search query
        text_query = {
            "size": k,  # Number of results to return
            "_source": {
                "excludes": ["doc_vector"]  # Exclude vector field from results
            },
            "query": {
                "match": {
                    "document": {
                        "query": search_text,
                        "analyzer": "nori_analyzer"
                    }
                }
            }
        }

        # Execute search
        response = client.search(
            index=index_name,
            body=text_query
        )


        documents = []
        if response.get("hits", {}).get("hits", []):
            search_results = normalize_search_results(response) 
            for res in search_results["hits"]["hits"]:
                source = res['_source']
                page_content = {k: source[k] for k in source if k != "table_summary_v"}
                metadata = {"id": res['_id']}
                score = res['_score']  # Get the score from the search result
                documents.append((Document(page_content=json.dumps(page_content, ensure_ascii=False), metadata=metadata), score))
        return documents  

    except Exception as e:
        print(f"Search error: {str(e)}")
        return None

#BM25 search의 score는 0.1 to 15.0 임으로 normazlie(0~1)를 해야함 (추후 vector search와 score 비교를 하기위해)
def normalize_search_results(search_results):
        hits = (search_results["hits"]["hits"])
        max_score = float(search_results["hits"]["max_score"])
        for hit in hits:
            hit["_score"] = float(hit["_score"]) / max_score
        search_results["hits"]["max_score"] = hits[0]["_score"]
        search_results["hits"]["hits"] = hits
        return search_results

In [65]:
#vector 유사도 검색 
def vector_search(client, embedding_vector, index_name="s3_explain", k=3):
    try:
        # KNN vector search query
        vector_query = {
            "size": k,
            "_source": {
                "excludes": ["doc_vector"]  # Exclude vector field from results
            },
            "query": {
                "knn": {
                    "doc_vector": {
                        "vector": embedding_vector,
                        "k": k
                    }
                }
            }
        }

        # Execute the search
        response = client.search(
            index=index_name,
            body=vector_query
        )

        documents = []
        for res in response["hits"]["hits"]:
            source = res['_source']
            page_content = {k: source[k] for k in source if k != "vector"}
            metadata = {"id": res['_id']}  # Add metadata with a unique identifier
            score = res['_score']  # Get the match score from the search result
            documents.append((Document(page_content=json.dumps(page_content, ensure_ascii=False), metadata=metadata), score))
        return documents
            

        return results

    except Exception as e:
        print(f"Search error: {str(e)}")
        return None

In [66]:
#노리를 통해 검색한 10개 + vector 유사도 기반으로 검색한 10개 총 20개를 정렬
def get_ensemble_results(doc_lists: List[List[Tuple[Document, float]]], weights: List[float], k: int = 5) -> List[Document]:
        hybrid_score_dic: Dict[str, float] = {}
        doc_map: Dict[str, Document] = {}
        
        # Weight-based adjustment
        for doc_list, weight in zip(doc_lists, weights):
            for doc, score in doc_list:
                doc_id = doc.metadata.get("id", doc.page_content)
                if doc_id not in hybrid_score_dic:
                    hybrid_score_dic[doc_id] = 0.0
                hybrid_score_dic[doc_id] += score * weight
                doc_map[doc_id] = doc
    
        sorted_docs = sorted(hybrid_score_dic.items(), key=lambda x: x[1], reverse=True)
        return [doc_map[doc_id] for doc_id, _ in sorted_docs[:k]]

In [67]:
#하이브리드 서치 코드 -> 노리를 통해 검색한 10개 + vector 유사도 기반으로 검색한 10개 총 20개에서 rerank를 통해 최종 10개만 선택한다
def retrieval_augmented(query , k =10): 
    
    #embedding 
    raw_embedding = get_embedding(query)
    #vector search 가져오기 
    vector=vector_search(opensearch_client, raw_embedding ,k =k)
    #lexical search 가져오기 
    lexical=simple_text_search(opensearch_client, query ,k =k)

    #rerank
    rerank_doc = get_ensemble_results(
            doc_lists=[vector, lexical],
            weights= [0.7, 0.3],
            k=k,
    )
    return rerank_doc
    

In [68]:
#결과 검색
result =retrieval_augmented("s3 에 대해 설명해줘")

In [69]:
result

[Document(metadata={'id': 'sagemaker_doc_1386'}, page_content='{"document": "경우 개별 디렉터리는 디렉터리 버킷의 최대 요청 속도를 지원하도록 설계되었습니다. 시스템이 균등\\n한 로드 분배를 위해 객체를 자동으로 분산하므로 최적의 성능을 달성하기 위해 키 접두사를 무작위화\\n고성능 워크로드 API 버전 2006-03-01 818", "source": "./s3-userguide.pdf", "page": 839, "total_pages": 3031}'),
 Document(metadata={'id': 'sagemaker_doc_331'}, page_content='{"document": "5. 삭제를 선택합니다.\\n6. 객체 삭제 페이지에서 삭제하기 위해 선택한 폴더와 객체의 이름이 지정된 객체에 나열되어 있는\\n지 확인합니다.\\n폴더 사용 API 버전 2006-03-01 432", "source": "./s3-userguide.pdf", "page": 453, "total_pages": 3031}'),
 Document(metadata={'id': 'sagemaker_doc_1117'}, page_content='{"document": "12. S3 인벤토리에 대한 권한을 부여하려면 정책 필드에서 다음 버킷 정책을 붙여 넣습니다.\\n세 가지 예제 값을 다음 값으로 바꿉니다.\\n자습서: 비디오 일괄 트랜스코딩 API 버전 2006-03-01 706", "source": "./s3-userguide.pdf", "page": 727, "total_pages": 3031}'),
 Document(metadata={'id': 'sagemaker_doc_416'}, page_content='{"document": "을 입력합니다.\\n표준 액세스 포인트와 마찬가지로 객체 Lambda 액세스 포인트에 대한 명명 규칙이 있습니다. 자\\n세한 내용은 액세스 포인트 이름 지정 규칙 섹션을 참조하세요.\\